In [43]:
import pandas as pd

# Polity5

In [44]:
print("Load Polity5 dataset")
polity_df = pd.read_excel('../datasets/raw/democracy/polity/POLITY5-PRC.xlsx', sheet_name='Data')

print("Rename columns")
polity_df.rename(columns={'Economy ISO3': 'country_iso', 'Economy Name': 'country_name'}, inplace=True)

polity_df['Indicator ID'] = polity_df['Indicator ID'].str.replace(r'^POLITY5\.PRC\.', '', regex=True)


print("Drop the columns: Attribute 1, Attribute 2, and Attribute 3")
polity_df.drop(columns=['Attribute 1', 'Attribute 2', 'Attribute 3'], inplace=True)

print("Set the years as identifiers instead of columns. Transpose the dataframe")
years = [str(year) for year in range(1776, 2019)]
polity_df_long = polity_df.melt(
    id_vars=['country_iso', 'country_name', 'Indicator ID'],
    value_vars=years,
    var_name='Year',
    value_name='Value'
)

print("Convert Year to numeric")
polity_df_long['Year'] = pd.to_numeric(polity_df_long['Year'])

print("Set the Indicator as the column")
polity_df_final = polity_df_long.pivot_table(
    index=['country_iso', 'country_name', 'Year'],
    columns='Indicator ID',
    values='Value'
).reset_index()


Load Polity5 dataset
Rename columns
Drop the columns: Attribute 1, Attribute 2, and Attribute 3
Set the years as identifiers instead of columns. Transpose the dataframe
Convert Year to numeric
Set the Indicator as the column


In [45]:
print("Verify that fields don't have NA values")
print(f"NA values in country_iso: {len(polity_df_final[polity_df_final['country_iso'].isna()])}")
print(f"NA values in autoc: {len(polity_df_final[polity_df_final['autoc'].isna()])}")
print(f"NA values in democ: {len(polity_df_final[polity_df_final['democ'].isna()])}")
print(f"NA values in exconst: {len(polity_df_final[polity_df_final['exconst'].isna()])}")
print(f"NA values in exrec: {len(polity_df_final[polity_df_final['exrec'].isna()])}")
print(f"NA values in parcomp: {len(polity_df_final[polity_df_final['parcomp'].isna()])}")
print(f"NA values in parreg: {len(polity_df_final[polity_df_final['parreg'].isna()])}")
print(f"NA values in polcomp: {len(polity_df_final[polity_df_final['polcomp'].isna()])}")
print(f"NA values in polity: {len(polity_df_final[polity_df_final['polity'].isna()])}")
print(f"NA values in polity2: {len(polity_df_final[polity_df_final['polity2'].isna()])}")
print(f"NA values in xconst: {len(polity_df_final[polity_df_final['xconst'].isna()])}")
print(f"NA values in xrcomp: {len(polity_df_final[polity_df_final['xrcomp'].isna()])}")
print(f"NA values in xropen: {len(polity_df_final[polity_df_final['xropen'].isna()])}")
print(f"NA values in xrreg: {len(polity_df_final[polity_df_final['xrreg'].isna()])}")


print(f"\nNA values in durable: {len(polity_df_final[polity_df_final['durable'].isna()])}")
print(f"NA values in durable after 1940: {len(polity_df_final[polity_df_final['durable'].isna() & polity_df_final["Year"].astype(int) > 1940])}")
print("Half of the records have NA values in durable predictor. Dropping the column.")

print(f"\nNA values in exrec after 1940: {len(polity_df_final[polity_df_final["exrec"].isna() & polity_df_final["Year"].astype(int) > 1940])}")
print("Some countries have NA values in exrec. Those are countries before 1940, which is already out of the scope of this analysis. Reduce the scope to 1940-2018")

print(f"\nNA values in polcomp after 1940: {len(polity_df_final[polity_df_final["polcomp"].isna() & polity_df_final["Year"].astype(int) > 1940])}")
print("Same happens for polcomp")

print(f"\npolity2 is a predictor built from polity")
# Only get columns polity and polity2
print(f"\nValues of polity when polity2 is NA:")
print(polity_df_final[["polity"]][polity_df_final["polity2"].isna()].value_counts())

print(f"Looking at the values of polity we have identified that polity2 is NA when polity is -66. Based on the codebook, 77 (interregnum) is set to 0 while -66 (interruption) is set to NA. We adjust 66 to 0.")
print(f"Records with polity = -88 should be prorated between years. Except when they're at the beginning of start of the dataset, as it's the case for India")
print("Dropping the polity2 column as the -66, -77, and -88 will be excluded at the time of the analysis")

Verify that fields don't have NA values
NA values in country_iso: 0
NA values in autoc: 0
NA values in democ: 0
NA values in exconst: 0
NA values in exrec: 183
NA values in parcomp: 0
NA values in parreg: 0
NA values in polcomp: 126
NA values in polity: 0
NA values in polity2: 228
NA values in xconst: 0
NA values in xrcomp: 0
NA values in xropen: 0
NA values in xrreg: 0

NA values in durable: 1165
NA values in durable after 1940: 0
Half of the records have NA values in durable predictor. Dropping the column.

NA values in exrec after 1940: 0
Some countries have NA values in exrec. Those are countries before 1940, which is already out of the scope of this analysis. Reduce the scope to 1940-2018

NA values in polcomp after 1940: 0
Same happens for polcomp

polity2 is a predictor built from polity

Values of polity when polity2 is NA:
polity
-66.0     223
-88.0       5
Name: count, dtype: int64
Looking at the values of polity we have identified that polity2 is NA when polity is -66. Based

In [46]:
print("Dropping the durable column")
# polity_df_final.drop(columns=['durable'], inplace=True)

print("Reduce the scope to 1940-2018")
polity_df_final = polity_df_final[polity_df_final["Year"].astype(int) > 1940]

print(f"Dropping the polity2 column")
# polity_df_final.drop(columns=['polity2'], inplace=True)

print("Transform the predictors from float64 to int64")
polity_df_final = polity_df_final.astype({'autoc': 'int64', 'democ': 'int64', 'exconst': 'int64', 'exrec': 'int64', 'parcomp': 'int64', 'parreg': 'int64', 'polcomp': 'int64', 'polity': 'int64', 'xconst': 'int64', 'xrcomp': 'int64', 'xrreg': 'int64', 'xropen': 'int64'})

print("Save the dataset")
polity_df_final.to_csv('../datasets/processed/democracy/polity/polity5.csv', index=False)

print(polity_df_final.info())

polity_df_final_stable = polity_df_final[~polity_df_final["democ"].isin([-66,-77,-88])]
print("Stats of records without -66, -77, and -88")
print(polity_df_final_stable.describe())

Dropping the durable column
Reduce the scope to 1940-2018
Dropping the polity2 column
Transform the predictors from float64 to int64
Save the dataset
<class 'pandas.core.frame.DataFrame'>
Index: 9845 entries, 141 to 16345
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_iso   9845 non-null   object 
 1   country_name  9845 non-null   object 
 2   Year          9845 non-null   int64  
 3   autoc         9845 non-null   int64  
 4   democ         9845 non-null   int64  
 5   durable       9800 non-null   float64
 6   exconst       9845 non-null   int64  
 7   exrec         9845 non-null   int64  
 8   parcomp       9845 non-null   int64  
 9   parreg        9845 non-null   int64  
 10  polcomp       9845 non-null   int64  
 11  polity        9845 non-null   int64  
 12  polity2       9715 non-null   float64
 13  xconst        9845 non-null   int64  
 14  xrcomp        9845 non-null   int64  
 15  xropen   

In [47]:
countries = polity_df_final["country_name"].unique()

print(f"Nr of countries: {len(countries)}")
print(f"From years {polity_df_final['Year'].min()} to {polity_df_final['Year'].max()}")


Nr of countries: 166
From years 1941 to 2018


# Freedom in the World

In [48]:
print("Load dataset")
freedom_df = pd.read_excel('../datasets/raw/democracy/freedom-world/All_data_FIW_2013-2024.xlsx', sheet_name='FIW13-25', header=1)

print("Rename columns")
freedom_df.rename(columns={'Country/Territory': 'country', 'Edition': 'year'}, inplace=True)

print("Drop the Region column. We're only interested in the country")
freedom_df.drop(columns=['Region'], inplace=True)

print("Drop the rows where the country is not a country")
freedom_df = freedom_df[freedom_df['C/T'] == 'c']

print("Drop the columns: Add Q, Add A, and C/T")
freedom_df.drop(columns=['Add Q', 'Add A', 'C/T'], inplace=True)

print("Rename columns with lower case and underscore")
freedom_df.columns = freedom_df.columns.str.strip().str.lower().str.replace(' ', '_')

print("Save the dataset")
freedom_df.to_csv('../datasets/processed/democracy/freedom-world/freedom-world.csv', index=False)

print(f"Rows with country or status with NA values: {len(freedom_df[freedom_df['country'].isna() | freedom_df['status'].isna()])}")
print("All the quantitative predictors are parsed as int64 as they don't have NA values")
print(freedom_df.info())

print("Describe the dataset")
print(freedom_df.describe())

freedom_df_countries = freedom_df["country"].unique()

print(f"Nr of countries: {len(freedom_df_countries)} - Countries: {', '.join(freedom_df_countries)}")

print(f"From years {freedom_df['year'].min()} to {freedom_df['year'].max()}")

Load dataset
Rename columns
Drop the Region column. We're only interested in the country
Drop the rows where the country is not a country
Drop the columns: Add Q, Add A, and C/T
Rename columns with lower case and underscore
Save the dataset
Rows with country or status with NA values: 0
All the quantitative predictors are parsed as int64 as they don't have NA values
<class 'pandas.core.frame.DataFrame'>
Index: 2535 entries, 1 to 2722
Data columns (total 40 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    2535 non-null   object
 1   year       2535 non-null   int64 
 2   status     2535 non-null   object
 3   pr_rating  2535 non-null   int64 
 4   cl_rating  2535 non-null   int64 
 5   a1         2535 non-null   int64 
 6   a2         2535 non-null   int64 
 7   a3         2535 non-null   int64 
 8   a          2535 non-null   int64 
 9   b1         2535 non-null   int64 
 10  b2         2535 non-null   int64 
 11  b3         2535 no

# Autocratic Regime Data

In [49]:
print("Load the Cases dataset")
autocratic_cases_df = pd.read_excel('../datasets/raw/democracy/autocratic/GWF Autocratic Regimes 1.2/GWF Autocratic Regimes.xlsx', sheet_name='Autocratic Regimes Case List', header=0)

print("Transform the dates to datetime in day/month/year format. By default Pandas uses month/day/year format.")
autocratic_cases_df['gwf_startdate'] = pd.to_datetime(autocratic_cases_df['gwf_startdate'], dayfirst=True, errors='coerce')
autocratic_cases_df['gwf_enddate'] = pd.to_datetime(autocratic_cases_df['gwf_enddate'], dayfirst=True, errors='coerce')

print("\nVerify NA values")
print(f"Rows with cowcode NA: {len(autocratic_cases_df[autocratic_cases_df['cowcode'].isna()])}")
print(f"Rows with gwf_country NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_country'].isna()])}")
print(f"Rows with gwf_startdate NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_startdate'].isna()])}")
print(f"Rows with gwf_enddate NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_enddate'].isna()])}")
print(f"Rows with gwf_startyr NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_startyr'].isna()])}")
print(f"Rows with gwf_endyr NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_endyr'].isna()])}")
print(f"Rows with gwf_subsreg NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_subsreg'].isna()])}")
print(f"Rows with gwf_howend NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_howend'].isna()])}")
print(f"Rows with gwf_violent NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_violent'].isna()])}")
print(f"Rows with gwf_regimetype NA: {len(autocratic_cases_df[autocratic_cases_df['gwf_regimetype'].isna()])}")

print("\nThe only case with Regime Type NA is Iran79, as it cannot be classified. We leave it as is. We don't want to exclude it from the analysis.")
print("\nEnd date is NA when the regime is still ongoing. We set it to -1 to be able to parse it as an integer")
autocratic_cases_df['gwf_endyr'] = autocratic_cases_df['gwf_endyr'].fillna(-1).astype(int)

print(autocratic_cases_df.info())

autocratic_cases_df.to_csv('../datasets/processed/democracy/autocratic/autocratic-regime-case-list.csv', index=False)

autocratic_cases_countries = autocratic_cases_df["gwf_country"].unique()

print(f"Autocratic Regimes - Nr of countries: {len(autocratic_cases_countries)}")
print(f"Autocratic Regimes - From years {autocratic_cases_df['gwf_startyr'].min()} to {autocratic_cases_df['gwf_endyr'].max()}")

Load the Cases dataset
Transform the dates to datetime in day/month/year format. By default Pandas uses month/day/year format.

Verify NA values
Rows with cowcode NA: 0
Rows with gwf_country NA: 0
Rows with gwf_startdate NA: 0
Rows with gwf_enddate NA: 0
Rows with gwf_startyr NA: 0
Rows with gwf_endyr NA: 57
Rows with gwf_subsreg NA: 0
Rows with gwf_howend NA: 0
Rows with gwf_violent NA: 0
Rows with gwf_regimetype NA: 1

The only case with Regime Type NA is Iran79, as it cannot be classified. We leave it as is. We don't want to exclude it from the analysis.

End date is NA when the regime is still ongoing. We set it to -1 to be able to parse it as an integer
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cowcode         280 non-null    int64         
 1   gwf_country     280 non-null    object        
 2   gwf_casename   

In [50]:
print("Load the TSCS Data")
print("The TSCS Data contains the autocratic regimes data over the years")
autocratic_years_df = pd.read_excel('../datasets/raw/democracy/autocratic/GWF Autocratic Regimes 1.2/GWF Autocratic Regimes.xlsx', sheet_name='TSCS data', header=0)

print("Transform the dates to datetime in day/month/year format. By default Pandas uses month/day/year format.")
autocratic_years_df['gwf_startdate'] = pd.to_datetime(autocratic_years_df['gwf_startdate'], dayfirst=True, errors='coerce')
autocratic_years_df['gwf_enddate'] = pd.to_datetime(autocratic_years_df['gwf_enddate'], dayfirst=True, errors='coerce')

print("\nVerify NA values")
print(f"Rows with cowcode NA: {len(autocratic_years_df[autocratic_years_df['cowcode'].isna()])}")
print(f"Rows with year NA: {len(autocratic_years_df[autocratic_years_df['year'].isna()])}")
print(f"Rows with gwf_country NA: {len(autocratic_years_df[autocratic_years_df['gwf_country'].isna()])}")
print(f"Rows with gwf_casename NA: {len(autocratic_years_df[autocratic_years_df['gwf_casename'].isna()])}")
print(f"Rows with gwf_startdate NA: {len(autocratic_years_df[autocratic_years_df['gwf_startdate'].isna()])}")
print(f"Rows with gwf_enddate NA: {len(autocratic_years_df[autocratic_years_df['gwf_enddate'].isna()])}")
print(f"Rows with gwf_spell NA: {len(autocratic_years_df[autocratic_years_df['gwf_spell'].isna()])}")
print(f"Rows with gwf_duration NA: {len(autocratic_years_df[autocratic_years_df['gwf_duration'].isna()])}")
print(f"Rows with gwf_fail NA: {len(autocratic_years_df[autocratic_years_df['gwf_fail'].isna()])}")
print(f"Rows with gwf_fail_subsregime NA: {len(autocratic_years_df[autocratic_years_df['gwf_fail_subsregime'].isna()])}")
print(f"Rows with gwf_fail_type NA: {len(autocratic_years_df[autocratic_years_df['gwf_fail_type'].isna()])}")
print(f"Rows with gwf_fail_violent NA: {len(autocratic_years_df[autocratic_years_df['gwf_fail_violent'].isna()])}")
print(f"Rows with gwf_regimetype NA: {len(autocratic_years_df[autocratic_years_df['gwf_regimetype'].isna()])}")
print(f"Rows with gwf_party NA: {len(autocratic_years_df[autocratic_years_df['gwf_party'].isna()])}")
print(f"Rows with gwf_personal NA: {len(autocratic_years_df[autocratic_years_df['gwf_personal'].isna()])}")
print(f"Rows with gwf_military NA: {len(autocratic_years_df[autocratic_years_df['gwf_military'].isna()])}")
print(f"Rows with gwf_monarch NA: {len(autocratic_years_df[autocratic_years_df['gwf_monarch'].isna()])}")

print("Similarly to the case list, Iran79 doesn't have a regime type. We leave it as is.")

print(autocratic_years_df.info())

autocratic_years_df.to_csv('../datasets/processed/democracy/autocratic/autocratic-regime-country-years.csv', index=False)

autocratic_years_countries = autocratic_years_df["gwf_country"].unique()

print(f"Autocratic Years - Nr of countries: {len(autocratic_years_countries)}")
print(f"Autocratic Years - From years {autocratic_years_df['year'].min()} to {autocratic_years_df['year'].max()}")

Load the TSCS Data
The TSCS Data contains the autocratic regimes data over the years
Transform the dates to datetime in day/month/year format. By default Pandas uses month/day/year format.

Verify NA values
Rows with cowcode NA: 0
Rows with year NA: 0
Rows with gwf_country NA: 0
Rows with gwf_casename NA: 0
Rows with gwf_startdate NA: 0
Rows with gwf_enddate NA: 0
Rows with gwf_spell NA: 0
Rows with gwf_duration NA: 0
Rows with gwf_fail NA: 0
Rows with gwf_fail_subsregime NA: 0
Rows with gwf_fail_type NA: 0
Rows with gwf_fail_violent NA: 0
Rows with gwf_regimetype NA: 31
Rows with gwf_party NA: 0
Rows with gwf_personal NA: 0
Rows with gwf_military NA: 0
Rows with gwf_monarch NA: 0
Similarly to the case list, Iran79 doesn't have a regime type. We leave it as is.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4591 entries, 0 to 4590
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   co

# Boix-Miller-Rosato Dichotomous Coding of Democracy

In [51]:
# Load CSV
print("Load the dichotomous democracy dataset")
dichotomous_df = pd.read_csv('../datasets/raw/democracy/dichotomous/democracy-v4.0.csv')
print(f"Initial number of records: {len(dichotomous_df)}")

print("Some countries aren't modern countries (E.g.: Bavaria), those countries have a NaN abbreviation. Dropping them")
dichotomous_df = dichotomous_df[dichotomous_df['abbreviation'].notna()]
print(f"Records after filtering: {len(dichotomous_df)}")

print("Some countries don't have democracy data (They should), filter those out")
dichotomous_df = dichotomous_df[dichotomous_df['democracy'].notna()]
print(f"Records after filtering: {len(dichotomous_df)}")

print("Transform columns to int")
dichotomous_df['democracy_trans'] = dichotomous_df['democracy_trans'].astype(int)
dichotomous_df['democracy_breakdowns'] = dichotomous_df['democracy_breakdowns'].astype(int)
dichotomous_df['democracy_duration'] = dichotomous_df['democracy_duration'].astype(int)
dichotomous_df['democracy_femalesuffrage'] = dichotomous_df['democracy_femalesuffrage'].astype(int)

print("Column democracy_omitteddata is allowed to have NA values, Codebook recommends to set it to 0 or other value if needed. We set it to -1")
dichotomous_df['democracy_omitteddata'] = dichotomous_df['democracy_omitteddata'].fillna(-1).astype(int)

print(dichotomous_df.info())

print(f"Rows with country NA: {len(dichotomous_df[dichotomous_df['country'].isna()])}")
print(f"Rows with abbreviation NA: {len(dichotomous_df[dichotomous_df['abbreviation'].isna()])}")
print(f"Rows with abbreviation_undp NA: {len(dichotomous_df[dichotomous_df['abbreviation_undp'].isna()])}")

dichotomous_countries = dichotomous_df["country"].unique()

print(f"Dichotomous Democracy - Nr of countries: {len(dichotomous_countries)}")
print(f"Dichotomous Democracy - From years {dichotomous_df['year'].min()} to {dichotomous_df['year'].max()}")

dichotomous_df.to_csv('../datasets/processed/democracy/dichotomous/dichotomous-democracy.csv', index=False)

Load the dichotomous democracy dataset
Initial number of records: 19775
Some countries aren't modern countries (E.g.: Bavaria), those countries have a NaN abbreviation. Dropping them
Records after filtering: 18889
Some countries don't have democracy data (They should), filter those out
Records after filtering: 18295
Transform columns to int
Column democracy_omitteddata is allowed to have NA values, Codebook recommends to set it to 0 or other value if needed. We set it to -1
<class 'pandas.core.frame.DataFrame'>
Index: 18295 entries, 0 to 19774
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country                   18295 non-null  object 
 1   ccode                     18295 non-null  int64  
 2   abbreviation              18295 non-null  object 
 3   abbreviation_undp         18295 non-null  object 
 4   year                      18295 non-null  int64  
 5   democracy                 1

# Lexical Index of Electoral Democracy (LIED)

In [52]:
print("Load Dataset")
lied_df = pd.read_excel('../datasets/raw/democracy/lied/LIED_6.6.xlsx', sheet_name='Sheet1', header=0)

print("\nDrop the regions column as it's not part of the codebook and it's included in the version 6.6 only")
lied_df.drop(columns=['region'], inplace=True)

print("\nDrop the rows where vdem is NA as it means it's not a country")
lied_df = lied_df[lied_df['vdem'].notna()]

print("\nTransform the year to int. There is one record with an invalid quote in the year -> ¨2023, we transform it to 2023") 
lied_df['year'] = lied_df['year'].replace('¨2023', '2023').astype(int)

print("\nVerify field types:")
print(lied_df.info())

print("\nOnly vdem is not parsed as int. Fixing it")
lied_df['vdem'] = lied_df['vdem'].astype(int)

lied_countries = lied_df["countryn"].unique()

print(f"LIED - Nr of countries: {len(lied_countries)}")
print(f"LIED - From years {lied_df['year'].min()} to {lied_df['year'].max()}")

lied_df.to_csv('../datasets/processed/democracy/lied/lied.csv', index=False)


Load Dataset

Drop the regions column as it's not part of the codebook and it's included in the version 6.6 only

Drop the rows where vdem is NA as it means it's not a country

Transform the year to int. There is one record with an invalid quote in the year -> ¨2023, we transform it to 2023

Verify field types:
<class 'pandas.core.frame.DataFrame'>
Index: 32276 entries, 0 to 32691
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   countryn                           32276 non-null  object 
 1   cow                                32276 non-null  int64  
 2   vdem                               32276 non-null  float64
 3   year                               32276 non-null  int64  
 4   male_suffrage                      32276 non-null  int64  
 5   female_suffrage                    32276 non-null  int64  
 6   executive_elections                32276 non-null  int64  
 7   le

# Pew Research Center - Global Religious Diversity

In [34]:
print("Load Dataset")
rdi_df = pd.read_excel('../datasets/raw/religion/pew-research-center-religion-diversity/religious-diversity-index.xlsx', sheet_name='Sheet1', header=0)

print("Verify field types and missing values")
print(rdi_df.info())

print("Rename columns to lowercase")
rdi_df.rename(columns={"Country": "country", "RDI": "rdi", "2010 Country Population": "population", "Percent Christian": "christian", 
                       "Percent Muslim": "muslim", "Percent Unaffiliated": "unaffiliated", "Percent Hindu": "hindu", 
                       "Percent Buddhist": "buddhist", "Percent Folk Religions": "folk", "Percent Other Religions": "other",
                       "Percent Jewish": "jewish"}, inplace=True)

print("Transform population to int")
rdi_df.loc[rdi_df['population'] == '< 10,000', 'population'] = 10000
rdi_df['population'] = rdi_df['population'].replace(',', '').astype(int)

print("Remove '%' symbol and convert to float")
for predictor in ['christian', 'muslim', 'unaffiliated', 'hindu', 'buddhist', 'folk', 'other', 'jewish']:
    rdi_df.loc[rdi_df[predictor] == '< 0.1%', predictor] = 0
    rdi_df[predictor] = rdi_df[predictor].replace('%', '').astype(float)

print("Verify field types")
print(rdi_df.info())

# rdi_df.describe()

rdi_df.to_csv('../datasets/processed/religion/pew-research-center-religion-diversity/religious-diversity-index.csv', index=False)



Load Dataset
Verify field types and missing values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Country                  232 non-null    object 
 1   RDI                      232 non-null    float64
 2   Percent Christian        232 non-null    object 
 3   Percent Muslim           232 non-null    object 
 4   Percent Unaffiliated     232 non-null    object 
 5   Percent Hindu            232 non-null    object 
 6   Percent Buddhist         232 non-null    object 
 7   Percent Folk Religions   232 non-null    object 
 8   Percent Other Religions  232 non-null    object 
 9   Percent Jewish           232 non-null    object 
 10  2010 Country Population  232 non-null    object 
dtypes: float64(1), object(10)
memory usage: 20.1+ KB
None
Rename columns to lowercase
Transform population to int
Remove '%' symbol and conve

# World Religion Project - National Religion Dataset

In [53]:
print("Load Dataset")
nat_relig_df = pd.read_csv('../datasets/raw/religion/world-religion-project/World Religion Project - National Religion Dataset.csv', header=0)

print("Verify field types and missing values")
print(nat_relig_df.info())

print("Rename all columns to lowercase")
nat_relig_df.columns = nat_relig_df.columns.str.lower()

print("Filter out countries where iso3 is NA, empty string, or doesn't have 3 characters. We'll exclude countries that don't exist anymore")
nat_relig_df = nat_relig_df[nat_relig_df['iso3'].notna() & (nat_relig_df['iso3'] != '') & (nat_relig_df['iso3'].str.len() == 3)]

print("judref and islmothr are empty in a few rows. Looking at it manually, those countries (Colombia and Myanmar) are not expected to have population of these religions. Set to 0")
nat_relig_df['judref'] = nat_relig_df['judref'].fillna(0)
nat_relig_df['islmothr'] = nat_relig_df['islmothr'].fillna(0)

print("For a given set of columns, remove the comma and convert to int")
for col in ["year", "chrsprot", "chrscat", "chrsorth", "chrsang", "chrsothr", "chrsgen", "judorth", "jdcons", "judref", "judothr", "judgen", "islmsun", "islmshi", "islmibd", "islmnat", "islmalw", "islmahm", "islmothr", "islmgen", "budmah", "budthr", "budothr", "budgen", "zorogen", "hindgen", "sikhgen", "shntgen", "bahgen", "taogen", "jaingen", "confgen", "syncgen", "anmgen", "nonrelig", "othrgen", "sumrelig", "pop"]:
    nat_relig_df[col] = nat_relig_df[col].astype(str).str.replace(',', '', regex=False).astype(int)

nat_relig_df["numiso"] = nat_relig_df["numiso"].astype(int)
nat_relig_df["isnatpct"] = nat_relig_df["isnatpct"].astype(float)


print("Verify field types")
print(nat_relig_df.info())

nat_relig_df.head()

nat_relig_df.to_csv('../datasets/processed/religion/world-religion-project/national-religion-dataset.csv', index=False)



Load Dataset
Verify field types and missing values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 87 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   YEAR      1995 non-null   object 
 1   COWCODE   1995 non-null   int64  
 2   ARDACODE  1995 non-null   int64  
 3   ISO3      1934 non-null   object 
 4   NUMISO    1942 non-null   float64
 5   COUNTRY   1995 non-null   object 
 6   CHRSPROT  1995 non-null   object 
 7   CHRSCAT   1995 non-null   object 
 8   CHRSORTH  1995 non-null   object 
 9   CHRSANG   1995 non-null   object 
 10  CHRSOTHR  1995 non-null   object 
 11  CHRSGEN   1995 non-null   object 
 12  JUDORTH   1995 non-null   object 
 13  JDCONS    1995 non-null   object 
 14  JUDREF    1993 non-null   object 
 15  JUDOTHR   1995 non-null   object 
 16  JUDGEN    1995 non-null   object 
 17  ISLMSUN   1995 non-null   object 
 18  ISLMSHI   1995 non-null   object 
 19  ISLMIBD   1995 non

# Georgetown University - Women Peace and Security Index

In [54]:
print("Load Dataset")
wps_df = pd.read_excel('../datasets/raw/women/georgetown/WPS-Index-data-final-cleaned.xlsx', sheet_name='Clean Data', header=0)

print("Change column names to lower case and spaces to underscores")
wps_df.columns = wps_df.columns.str.lower().str.replace(' ', '_').str.replace(',', '')

print("Verify field types and missing values")
print(wps_df.info())

# wps_df.describe()

wps_df.to_csv('../datasets/processed/women/georgetown/women-peace-and-security-index.csv', index=False)

Load Dataset
Change column names to lower case and spaces to underscores
Verify field types and missing values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   wps_rank                            177 non-null    int64  
 1   country                             177 non-null    object 
 2   women_peace_and_security_index      177 non-null    float64
 3   education                           177 non-null    float64
 4   employment                          177 non-null    float64
 5   financial_inclusion                 177 non-null    float64
 6   cell_phone_use                      177 non-null    float64
 7   parliamentary_representation        177 non-null    float64
 8   absence_of_legal_discrimination     177 non-null    float64
 9   access_to_justice                   177 non-null    float64
 10 

# World Bank Group - Women, Business and the Law Historical Data

In [55]:
print("Load Dataset")
wbl_df = pd.read_excel('../datasets/raw/women/world-bank/WBL2024-1-0-Historical-Panel-Data.xlsx', sheet_name='WBL Panel 2024', header=0)

print("Drop and Rename columns")
wbl_df.drop(columns={"Economy Code","Region","Income Group"}, inplace=True)
wbl_df.rename(columns={"Economy": "country", "Report Year": "year", "WBL INDEX": "wbl_index", "MOBILITY": "mobility", "WORKPLACE": "workplace", "PAY": "pay", "MARRIAGE": "marriage", "PARENTHOOD": "parenthood", "ENTREPRENEURSHIP": "entrepreneurship", "ASSETS": "assets", "PENSION": "pension"}, inplace=True)

print("Rename qualitative columns")
question_columns = {
    "Can a woman choose where to live in the same way as a man?": "woman_choose_residence",
    "Can a woman travel outside her home in the same way as a man?": "woman_travel_outside_home",
    "Can a woman apply for a passport in the same way as a man?": "woman_apply_passport",
    "Can a woman travel outside the country in the same way as a man?": "woman_travel_abroad",
    "Can a woman get a job in the same way as a man?": "woman_get_job",
    "Does the law prohibit discrimination in employment based on gender?": "law_prohibits_gender_discrimination",
    "Is there legislation on sexual harassment in employment?": "law_sexual_harassment",
    "Are there criminal penalties or civil remedies for sexual harassment in employment?": "penalties_sexual_harassment",
    "Does the law mandate equal remuneration for work of equal value?": "law_equal_pay",
    "Can a woman work at night in the same way as a man?": "woman_work_night",
    "Can a woman work in a job deemed dangerous in the same way as a man?": "woman_work_dangerous",
    "Can a woman work in an industrial job in the same way as a man?": "woman_work_industrial",
    "Is the law free of legal provisions that require a married woman to obey her husband?": "no_obey_husband_law",
    "Can a woman be head of household in the same way as a man?": "woman_head_household",
    "Is there legislation specifically addressing domestic violence?": "law_domestic_violence",
    "Can a woman obtain a judgment of divorce in the same way as a man?": "woman_divorce_rights",
    "Does a woman have the same rights to remarry as a man?": "woman_remarry_rights",
    "Is paid leave of at least 14 weeks available to mothers?": "paid_maternity_leave_14weeks",
    "Length of paid maternity leave": "paid_maternity_leave_length",
    "Does the government administer 100 percent of maternity leave benefits?": "govt_pays_maternity_leave",
    "Is there paid leave available to fathers?": "paid_paternity_leave",
    "Length of paid paternity leave": "paid_paternity_leave_length",
    "Is there paid parental leave?": "paid_parental_leave",
    "Shared days": "parental_leave_shared_days",
    "Days for the mother": "parental_leave_mother_days",
    "Days for the father": "parental_leave_father_days",
    "Is dismissal of pregnant workers prohibited?": "protect_pregnant_workers",
    "Does the law prohibit discrimination in access to credit based on gender?": "law_credit_gender_discrimination",
    "Can a woman sign a contract in the same way as a man?": "woman_sign_contract",
    "Can a woman register a business in the same way as a man?": "woman_register_business",
    "Can a woman open a bank account in the same way as a man?": "woman_open_bank_account",
    "Do women and men have equal ownership rights to immovable property?": "equal_property_rights",
    "Do sons and daughters have equal rights to inherit assets from their parents?": "equal_inheritance_children",
    "Do male and female surviving spouses have equal rights to inherit assets?": "equal_inheritance_spouses",
    "Does the law grant spouses equal administrative authority over assets during marriage?": "equal_asset_admin_marriage",
    "Does the law provide for the valuation of nonmonetary contributions?": "value_nonmonetary_contributions",
    "Is the age at which women and men can retire with full pension benefits the same?": "equal_pension_age_full",
    "Is the age at which women and men can retire with partial pension benefits the same?": "equal_pension_age_partial",
    "Is the mandatory retirement age for women and men the same?": "equal_retirement_age",
    "Are periods of absence due to childcare accounted for in pension benefits?": "pension_credit_childcare"
}

wbl_df.rename(columns=question_columns, inplace=True)

print("Transform qualitative columns to booleans")
qualitative_columns = [
"woman_choose_residence",
"woman_travel_outside_home",
"woman_apply_passport",
"woman_travel_abroad",
"woman_get_job",
"law_prohibits_gender_discrimination",
"law_sexual_harassment",
"penalties_sexual_harassment",
"law_equal_pay",
"woman_work_night",
"woman_work_dangerous",
"woman_work_industrial",
"no_obey_husband_law",
"woman_head_household",
"law_domestic_violence",
"woman_divorce_rights",
"woman_remarry_rights",
"paid_maternity_leave_14weeks",
"govt_pays_maternity_leave",
"paid_paternity_leave",
"paid_parental_leave",
"protect_pregnant_workers",
"law_credit_gender_discrimination",
"woman_sign_contract",
"woman_register_business",
"woman_open_bank_account",
"equal_property_rights",
"equal_inheritance_children",
"equal_inheritance_spouses",
"equal_asset_admin_marriage",
"value_nonmonetary_contributions",
"equal_pension_age_full",
"equal_pension_age_partial",
"equal_retirement_age",
"pension_credit_childcare"
]

wbl_df[qualitative_columns] = wbl_df[qualitative_columns].applymap(lambda x: True if x == 'Yes' else False)

wbl_df.info()

wbl_df.to_csv('../datasets/processed/women/world-bank/women-business-and-the-law-historical.csv', index=False)

Load Dataset
Drop and Rename columns
Rename qualitative columns
Transform qualitative columns to booleans
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10260 entries, 0 to 10259
Data columns (total 52 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              10260 non-null  object 
 1   ISO Code                             10260 non-null  object 
 2   year                                 10260 non-null  int64  
 3   wbl_index                            10260 non-null  float64
 4   mobility                             10260 non-null  int64  
 5   woman_choose_residence               10260 non-null  bool   
 6   woman_travel_outside_home            10260 non-null  bool   
 7   woman_apply_passport                 10260 non-null  bool   
 8   woman_travel_abroad                  10260 non-null  bool   
 9   workplace                            10260 non-null  i

/var/folders/3c/bf3091xd4h9fj5rvqhp6_05r0000gq/T/ipykernel_17318/1343447439.py:93: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  wbl_df[qualitative_columns] = wbl_df[qualitative_columns].applymap(lambda x: True if x == 'Yes' else False)
